# 1. import packages

In [1]:
from google.cloud import bigquery
from utils_hack import get_eeat_prompts, run_assessment,prompt_api
import pandas as pd
import json

# 2. Set up google cloud and visualize databases

In [2]:
from google.cloud import bigquery

# Connect to BigQuery
PROJECT_ID = "dbce-cmap-seo-dev-9e60"
DATASET_ID = "hcktn_f_ai_search_insights"

client = bigquery.Client(project=PROJECT_ID)
print(f"Connected to project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Connected to project: dbce-cmap-seo-dev-9e60
Dataset: hcktn_f_ai_search_insights


In [3]:
# List tables in the dataset
dataset_ref = client.dataset(DATASET_ID)
tables = list(client.list_tables(dataset_ref))

print(f"Tables in {DATASET_ID}:")
for table in tables:
    print(f"  - {table.table_id}")

Tables in hcktn_f_ai_search_insights:
  - ai_readiness_score_timeline
  - ai_readiness_score_timeline_copy
  - all_checkpoints_storage
  - api_markdowns
  - crawlability_issues_checkpoint
  - crawlability_issues_checkpoint_backup_11_07
  - cwv_checkpoint
  - domain_authority
  - domain_authority_checkpoint
  - eeat_score_timeline
  - eeat_score_timeline_copy
  - image_alt_tags_checkpoint
  - image_alt_tags_checkpoint_copy2
  - js_issue_checkpoint
  - structured_data_checkpoint
  - url_prompts
  - url_status
  - urls_page_types
  - urls_page_types_api
  - urls_page_types_api_copy_backup
  - urls_page_types_copy_backup
  - urls_prompt_checkpoints_ref
  - urls_prompt_checkpoints_results
  - urls_prompt_results
  - urls_prompt_results_api
  - urls_prompt_results_backup_11_07


# 3. Initiate analysis for the data

* The hackathon organizer gave us a set of prepared prompts that we can use to analyze and generate an assesment for each input.
* It is recommended to use the prompts which have a "_m" in as they analyze websites and general text
* If the output is suspicious we can fix the parsers in the 

In [4]:
df_prompt = pd.json_normalize(get_eeat_prompts()['data']).query('promptName!="content_angle"')

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
df_prompt.shape

(22, 9)

In [6]:
df_prompt.sample(5)

,promptName,tags,promptText,variables,temperature,model,responseModality,additionalParams,type
0,author_info_m,[EEAT],# Task\n Find information about article Author...,[{markdown}],0.7,GPT-41-MINI-2025-04-14,text,[],single
21,page_purpose_m,[EEAT],# Task\nFigure out page purpose based on URL p...,[{Markdown}],0.7,GPT-4O-MINI-2024-07-18,text,[],single
16,expertise,[EEAT],# Task\n Audit the content in terms of Exp...,"[{content}, {title}]",0.7,GPT-4O-MINI-2024-07-18,text,[],single
8,experience_m,[EEAT],# Task\n Audit the Content in terms of Expe...,[{Markdown}],0.8,GEMINI-2.0-FLASH-LITE-001,text,[],single
18,title_quality,[EEAT],# Task\nAssess TITLE quality based on Content ...,"[{content}, {title}]",0.7,O3-MINI-2025-01-31,text,[],single


In [26]:
sample_test =  { 'content':"""Hi my name is sarkis and I will take to you about my life. This is a link to my CV www.sarkisprowebsite.com, whereas this link talks about my personal life
                                     www.sarkisinsider.com.I'm from Lebanon and I have lived there for most of my life years. I studied there and I did my masters in Astrophysics. Then I went to armenia to do
                                     data science internships. Then I came to poland.
                                     """,
                 'markdown':"""Hi my name is sarkis and I will take to you about my life. This is a link to my CV www.sarkisprowebsite.com, whereas this link talks about my personal life
                                     www.sarkisinsider.com.I'm from Lebanon and I have lived there for most of my life years. I studied there and I did my masters in Astrophysics. Then I went to armenia to do
                                     data science internships. Then I came to poland.
                                     """,
                'url': "https://en.wikipedia.org/wiki/Astronomy",

                
                'title': "Life of Sarkis",

                'html code': """
                            <!DOCTYPE html>
                            <html lang="en">
                            <head>
                                <meta charset="UTF-8">
                                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                                <title>About Sarkis</title>
                                <style>
                                    body {
                                        font-family: Arial, sans-serif;
                                        line-height: 1.6;
                                        margin: 20px;
                                    }
                                    h1 {
                                        color: #333;
                                    }
                                    p {
                                        margin: 10px 0;
                                    }
                                    a {
                                        color: #007BFF;
                                        text-decoration: none;
                                    }
                                    a:hover {
                                        text-decoration: underline;
                                    }
                                </style>
                            </head>
                            <body>
                                <h1>About Me</h1>
                                <p>Hi, my name is <strong>Sarkis</strong> and I will talk to you about my life.</p>
                                
                                <p>This is a link to my <a href="http://www.sarkisprowebsite.com" target="_blank">CV</a>, whereas this link talks about my personal life: <a href="http://www.sarkisinsider.com" target="_blank">Personal Life</a>.</p>
                                
                                <p>I was born in <strong>Syria</strong> and I grew up in <strong>Lebanon</strong> and I have <strong>Armenian origins</strong>. I studied there and completed my master's degree in <strong>Astrophysics</strong>.</p>
                                
                                <p>Then I went to <strong>Armenia</strong> to do data science internships. After that, I came to <strong>Poland</strong> for more data science opportunities.</p>
                            </body>
                            </html>""" 

                } 

In [29]:
total_res = {} 
for index in  df_prompt.query("promptName=='ymyl'").itertuples():
    prompt_name = index.promptName
    input_type = index.variables[0].replace("{","").replace("}","") 
    input_title = ""
    #if len(index.variables) == 2: 
    #    input_title = sample_test["title"]
        
    total_res[prompt_name] = prompt_api(input_value = sample_test['url'],
                         input_type = input_type,
                         prompt_type = prompt_name,
                         title = input_title)

/home/kassounians/projects/hackathon/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pgcloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[INFO]: LLM API response: ymyl -> 200


In [31]:
 print(total_res[prompt_name]['assistant'][0]['ymyl'])

Output:
{
  "question_1": {
    "answer": "Yes, a careful person would seek out experts or trusted sources when dealing with astronomical data or implications.",
    "rating": true
  },
  "question_2": {
    "answer": "Yes, the topic of astronomy would likely be one that people would not only consult casually with friends.",
    "rating": true
  },
  "question_3": {
    "answer": "Yes, significant harm could result from inaccurate information, especially regarding space safety or understanding celestial events.",
    "rating": true
  },
  "question_4": {
    "answer": "Yes, poor advice on astronomical events could lead to misunderstanding risks associated with space phenomena.",
    "rating": true
  },
  "question_5": {
    "answer": "Yes, astronomy can significantly impact people and society, especially in terms of understanding natural events and their implications.",
    "rating": true
  },
  "question_6": {
    "answer": "Yes, if misinformation about astronomical events were widely